In [2]:
from selenium.webdriver.common.by import By
from __future__ import unicode_literals
from fake_useragent import UserAgent
from selenium import webdriver
from bs4 import BeautifulSoup
from yt_dlp import YoutubeDL

import urllib.request
import time
import os

# Step 1: Get Songs

In [7]:
url = "https://www.billboard.com/charts/greatest-hot-100-singles/"

# writes raw HTML to file
with open("billboard.txt", "w") as file:
    file.write(urllib.request.urlopen(url).read().decode("utf-8"))        

# parses raw HTML
with open("billboard.txt", "r") as file:
    html = file.read()
soup = BeautifulSoup(html, "html.parser")

sp = soup.get_text().replace("\n", "")
print(sp)

Greatest Songs of All Time | Hot 100 - Billboard     ×   ×			Skip to main content		 Billboard	Charts	Music	Video	Shop	Awards	Business		LOGIN	LoginClick to Expand Search Input 			Account				Manage Account		Log Out		AccountOpen menuSearchClick to Expand Search Input  Billboard BillboardSearchClick to Expand Search Input 		LOGIN	Login			Account				Manage Account		Log Out		AccountSearchClick to Expand Search Input 	Charts	Music	Video	Shop	Awards	Business	Billboard Hot 100	Chart Beat	Latin Music Week	Get Up Anthems by Tres	TikTok Billboard Top 50	Family	Global News	EspañolOpen menuSearchClick to Expand Search Input  Billboard BillboardSearchClick to Expand Search Input 		LOGIN	Login			Account				Manage Account		Log Out		AccountSearchClick to Expand Search Input 	Charts	Music	Video	Shop	Awards	Business   			Greatest of all time					Greatest of All Time Hot 100 Songs			All Charts		All Charts	Info			Greatest of All Time Hot 100 Songs		These all-time rankings are based on actual performance on 

In [4]:
# [rank-1] [song title] [artist name]
rankings = []

i = 1
while len(rankings) < 100:
    ind = sp.find(f"\t{i}\t")
    rest = sp[ind:].lstrip("\t")
    line = rest[:rest.find(f"{i+1}")]
    
    num = line[:len(str(i))]
    
    # line now starts at song title
    line = line[len(str(i)):].lstrip("\t")
    songTitle = line[:line.find("\t")]
    
    # line now starts at artist name
    line = line[len(songTitle):].lstrip("\t")
    artistName = line[:line.find("\t")]

    rankings.append([songTitle, artistName])

    i += 1

# for i, entry in enumerate(rankings):
#     song, artist = entry
#     print(f"{i}: {song}\t{artist}")
rankings[-1][1] = "Carly Simon" # fixing an error
print(rankings[-1])

["You're So Vain", 'Carly Simon']


## Web Scraping

In [ ]:
# start driver
options = webdriver.SafariOptions()

ua = UserAgent(browsers=["safari"])
options.add_argument(f"user-agent={ua.random}")
options.add_argument("--incognito")

driver = webdriver.Safari(options=options)
driver.maximize_window()

### Get link to YouTube video

In [39]:
# test: get song from top of billboard top 100
song, artist = rankings[78]
driver.get(f"https://www.youtube.com/results?search_query={song}+{artist}")

In [40]:
# get link to video
time.sleep(20)
href = None
videos = driver.find_elements(
            By.TAG_NAME, 
            "ytd-thumbnail"
        )
while len(videos) > 0 and not href:
    href = videos[0].get_attribute("href")
    videos = videos[1:]
    
if not href:
    videos = driver.find_elements(
        By.ID, 
        "video-title"
    )
    while len(videos) > 0 and not href:
        href = videos[0].get_attribute("href")
        videos = videos[1:]
        
if not href:
    videos = driver.find_elements(
        By.ID, 
        "movie_player"
    )
    while len(videos) > 0 and not href:
        href = videos[0].get_attribute("href")
        videos = videos[1:]

print(href)

https://www.youtube.com/watch?v=Xc1uQI2XmZs&pp=ygUuQmVjYXVzZSBJIExvdmUgWW91IChUaGUgUG9zdG1hbiBTb25nKSBTdGV2aWUgQg%3D%3D


### Get key of song

In [68]:
driver.get("https://tunebat.com")
time.sleep(10)

# type song and artist into search bar
search = driver.find_element(
    By.XPATH,
    "//input[@aria-label='Song search field']"
)
search.send_keys(f"{song} {artist}")

time.sleep(1)

# click search button
button = driver.find_element(
    By.XPATH,
    "//i[@aria-label='Search']"
)
button.click()

time.sleep(5)

# get info on song
song_info = driver.find_elements(
    By.CLASS_NAME,
    "lAjUd"
)
key, bpm = [item.text for item in song_info[:2]]
print(key, bpm)

C# Major 171


### Download YouTube video using youtube_dl

In [94]:
kebab_artist = '-'.join(artist.split())
kebab_song = '-'.join(song.split())
dl_path = os.join(os.getcwd(),
                  "songs",
                  str(kebab_artist),
                  str(kebab_song),
                  str(key),
                  str(bpm),
                  ".mp3"
                 )

with YoutubeDL({
     'extract_audio': True,
     'format': 'bestaudio',
     'outtmpl': dl_path
    }) as audio:
    
    audio.download(href)

[youtube] Extracting URL: https://www.youtube.com/watch?v=4NRXx6U8ABQ&pp=ygUaQmxpbmRpbmcgTGlnaHRzIFRoZSBXZWVrbmQ%3D
[youtube] 4NRXx6U8ABQ: Downloading webpage
[youtube] 4NRXx6U8ABQ: Downloading ios player API JSON
[youtube] 4NRXx6U8ABQ: Downloading player b7240855


         n = rgklxtY-d8uPBxe4 ; player = https://www.youtube.com/s/player/b7240855/player_ias.vflset/en_US/base.js
         n = SM7HnJ52tMRYmB4r ; player = https://www.youtube.com/s/player/b7240855/player_ias.vflset/en_US/base.js


[youtube] 4NRXx6U8ABQ: Downloading m3u8 information
[info] 4NRXx6U8ABQ: Downloading 1 format(s): 140
[download] Destination: /Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/The-Weeknd_Blinding-Lights_C# Major_171.mp3
[download] 100% of    4.05MiB in 00:00:00 at 5.55MiB/s   


# Step 1.1: Convert to .m4a (.mp3 is incompatible with librosa)

In [147]:
# find which files did not convert to .m4a
converted = {}
need_to_convert = []

for file in os.listdir(os.path.join("instrumentals")):
    if file[0] != ".":
        filename = file[:-4]
        extension = file[-4:]
        if filename not in converted:
            converted.update({filename : [extension]})
        else:
            converted[filename].append(extension)
            
for key, value in converted.items():
    if len(value) < 2:
        print(f"{key} did not convert")
        need_to_convert.append(key+value[0])

Marvin-Gaye_Lets-Get-It-On_E♭-Major_167_instrumental did not convert


In [148]:
# convert remaining files in "/instrumentals" to .m4a
for file in need_to_convert:
    filepath = os.path.join(os.getcwd(), "instrumentals", file)
    os.system(f"ffmpeg -i {filepath} -c:a aac -b:a 192k {filepath[:-4]}.m4a")

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

In [128]:
# convert all audio files in "/songs" from .mp3 to .m4a
for file in os.listdir(os.path.join("songs")):
    if file[0] != ".":
        filepath = os.path.join(os.getcwd(), "songs", file)
        os.system(f"ffmpeg -i {filepath} -c:a aac -b:a 192k {filepath[:-4]}.m4a")

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Kanye-West-Featuring-Jamie-Foxx_Gold-Digger_F#-Minor_185.m4a':
  Metadata:
    major_brand     : M4A 
    minor_version   : 512
    compatible_brands: M4A isomiso2
    encoder         : Lavf61.7.100
  Duration: 00:03:41.08, start: 0.000000, bitrate: 197 kb/s
  Stream #0:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 196 kb/s (default)
      Metadata:
        handler_name    : ISO Media file produced by Google Inc.
        vendor_id       : [0][0][0][0]
File '/Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Kanye-West-Featuring-Jamie-Foxx_Gold-Digger_F#-Minor_185.m4a' already exists. Overwrite? [y/N] Not overwriting - exiting
Error opening output file /Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Kanye-West-Featuring-Jamie-Foxx_Gold-Digger_F#-Minor_185.m4a.
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

...

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Kenny-Rogers_Lady_E♭-Minor_137.m4a':
  Metadata:
    major_brand     : M4A 
    minor_version   : 512
    compatible_brands: M4A isomiso2
    encoder         : Lavf61.7.100
  Duration: 00:03:51.39, start: 0.000000, bitrate: 194 kb/s
  Stream #0:0[0x1](eng): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 193 kb/s (default)
      Metadata:
        handler_name    : ISO Media file produced by Google Inc.
        vendor_id       : [0][0][0][0]
File '/Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Kenny-Rogers_Lady_E♭-Minor_137.m4a' already exists. Overwrite? [y/N] Not overwriting - exiting
Error opening output file /Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Kenny-Rogers_Lady_E♭-Minor_137.m4a.
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/u

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Survivor_Eye-Of-The-Tiger_C-Minor_109.m4a':
  Metadata:
    major_brand     : M4A 
    minor_version   : 512
    compatible_brands: M4A isomiso2
    encoder         : Lavf61.7.100
  Duration: 00:04:04.88, start: 0.000000, bitrate: 195 kb/s
  Stream #0:0[0x1](eng): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 194 kb/s (default)
      Metadata:
        handler_name    : ISO Media file produced by Google Inc.
        vendor_id       : [0][0][0][0]
File '/Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Survivor_Eye-Of-The-Tiger_C-Minor_109.m4a' already exists. Overwrite? [y/N] Not overwriting - exiting
Error opening output file /Users/reallytherealest/Desktop/CS/Testing/ChordGuessr/songs/Survivor_Eye-Of-The-Tiger_C-Minor_109.m4a.
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  confi

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

In [120]:
# convert all audio files in "/instrumentals" from .mp3 to .m4a
for file in os.listdir(os.path.join("instrumentals")):
    if file[0] != ".":
        filepath = os.path.join(os.getcwd(), "instrumentals", file)
        os.system(f"ffmpeg -i {filepath} -c:a aac -b:a 192k {filepath[:-4]}.m4a")

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enab

[out#0/ipod @ 0x7fdeea904f80] video:0KiB audio:5264KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.725355%
size=    5302KiB time=00:03:41.70 bitrate= 195.9kbits/s speed=30.3x    
[aac @ 0x7fdeea905e80] Qavg: 1219.211
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib

[out#0/ipod @ 0x7fb901f052c0] video:0KiB audio:5360KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.714969%
size=    5399KiB time=00:03:42.60 bitrate= 198.7kbits/s speed=12.1x    
[aac @ 0x7fb9038058c0] Qavg: 635.865
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx

[out#0/ipod @ 0x7ff6dda04400] video:0KiB audio:6280KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.720735%
size=    6325KiB time=00:04:23.68 bitrate= 196.5kbits/s speed=14.2x    
[aac @ 0x7ff6dda05300] Qavg: 2021.112
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib

[out#0/ipod @ 0x7fb4ad706080] video:0KiB audio:5959KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.762501%
size=    6004KiB time=00:04:24.82 bitrate= 185.7kbits/s speed=27.2x    
[aac @ 0x7fb4ae804bc0] Qavg: 4033.299
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib

...

[out#0/ipod @ 0x7f8052f05d40] video:0KiB audio:5165KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.780347%
size=    5206KiB time=00:03:54.33 bitrate= 182.0kbits/s speed=23.3x    
[aac @ 0x7f8054204080] Qavg: 5996.469
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib

[out#0/ipod @ 0x7fcfedf05780] video:0KiB audio:4982KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.782155%
size=    5021KiB time=00:03:46.46 bitrate= 181.6kbits/s speed=34.3x    
[aac @ 0x7fcfedf0a0c0] Qavg: 7181.426
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib

[out#0/ipod @ 0x7fcc0d00d300] video:0KiB audio:5614KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.764591%
size=    5657KiB time=00:04:09.86 bitrate= 185.5kbits/s speed=34.2x    
[aac @ 0x7fcc0d009b80] Qavg: 4619.427
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib

[out#0/ipod @ 0x7f99a0b0a600] video:0KiB audio:3521KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.805000%
size=    3550KiB time=00:02:43.42 bitrate= 177.9kbits/s speed=32.4x    
[aac @ 0x7f99a0b07840] Qavg: 7061.271
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib

[out#0/ipod @ 0x7fd833a04600] video:0KiB audio:6433KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.707329%
size=    6478KiB time=00:04:25.10 bitrate= 200.2kbits/s speed=19.6x    
[aac @ 0x7fd833a08b40] Qavg: 1081.272
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-lib